# NP Semantic Segmentation

Let's import all the relevant classes

In [1]:
import os
os.chdir(os.path.dirname(os.path.dirname(os.getcwd())))
os.getcwd()

'/Users/ayaccobi/ai-lab-nlp'

In [1]:
from examples.np_semantic_segmentation.data import NpSemanticSegData
from examples.np_semantic_segmentation.preprocess_tratz2011 import *
from examples.np_semantic_segmentation.data import *
from nlp_architect.models.np_semantic_segmentation import NpSemanticSegClassifier

ModuleNotFoundError: No module named 'examples.np_semantic_segmentation'

## Preparing the data

The first step is the download the dataset into a folder. You can download Tratz 2011 et al. dataset from the following link: [Tratz 2011 Dataset](https://vered1986.github.io/papers/Tratz2011_Dataset.tar.gz). Is also available in [here](https://www.isi.edu/publications/licensed-sw/fanseparser/index.html). (The terms and conditions of the data set license apply. Intel does not grant any rights to the data files or database).

After downloading and unzipping the dataset, the following method will labels some portion of the data, and will output two `.csv` files that will assist us to train and evaluate the trained model.

In [3]:
dataset_path = '/Users/ayaccobi/nlp_architect/data/np_semantic_segmentation'
preprocess_tratz_2011(dataset_path)

CSV file is saved in /Users/ayaccobi/nlp_architect/data/np_semantic_segmentation/tratz2011_coarse_grained_random/train.csv
CSV file is saved in /Users/ayaccobi/nlp_architect/data/np_semantic_segmentation/tratz2011_coarse_grained_random/val.csv


Once the dataset is saved and labeled we need to vectories the data:

In [3]:
# labeled_data_path is the output of preprocess_tratz_2011()
labeled_train_data_path = '/Users/ayaccobi/nlp_architect/data/np_semantic_segmentation/tratz2011_coarse_grained_random/train.csv'
labeled_val_data_path = '/Users/ayaccobi/nlp_architect/data/np_semantic_segmentation/tratz2011_coarse_grained_random/val.csv'
word2vec_path = '/Users/ayaccobi/workspace/word_embeddings/GoogleNews-vectors-negative300.bin.gz'
# output_path is location to save the vectors
train_output_path = '/Users/ayaccobi/nlp_architect/data/np_semantic_segmentation/prepared_data_train.csv'
val_output_path = '/Users/ayaccobi/nlp_architect/data/np_semantic_segmentation/prepared_data_val.csv'
http_proxy = None
https_proxy = None

In [5]:

prepare_data(labeled_train_data_path, train_output_path, word2vec_path, http_proxy, https_proxy)
prepare_data(labeled_val_data_path, val_output_path, word2vec_path, http_proxy, https_proxy)

Start loading NLTK Collocation scoring (this might take a while...)
Start loading Word2Vec model (this might take a while...)
Finish loading feature extraction services


100%|██████████| 324/324 [01:08<00:00,  4.72it/s]


prepared data CSV file is saved in /Users/ayaccobi/nlp_architect/data/np_semantic_segmentation/prepared_data_train.csv
Start loading NLTK Collocation scoring (this might take a while...)
Start loading Word2Vec model (this might take a while...)


Process ForkPoolWorker-5:
Process ForkPoolWorker-2:
Process ForkPoolWorker-3:
Process ForkPoolWorker-6:
Process ForkPoolWorker-8:
Process ForkPoolWorker-4:
Process ForkPoolWorker-1:
Process ForkPoolWorker-9:
Process ForkPoolWorker-10:
Process ForkPoolWorker-7:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/Users/ayaccobi/anaconda3/envs/fresh_env/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/Users/ayaccobi/anaconda3/envs/fresh_env/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/Users/ayaccobi/anaconda3/envs/fresh_env/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.ru

KeyboardInterrupt: 

We now need to load the data into NpSemanticSegmentation object.

In [4]:
data_set = NpSemanticSegData(train_output_path, train_to_test_ratio=0.8)

## Build the model

In [5]:
    model_file_path = 'np_semantic_segmentation.h5'
    model = NpSemanticSegClassifier(num_epochs=200, callback_args=None)
    input_dim = data_set.train_set_x.shape[1]
    model.build(input_dim)
    model.fit(data_set.train_set)


Epoch 1/200
 - 0s - loss: 1.3596 - binary_accuracy: 0.5465 - precision_score: 0.1740 - recall_score: 0.3135 - f1: 0.2238
Epoch 2/200
 - 0s - loss: 1.2109 - binary_accuracy: 0.6899 - precision_score: 0.2617 - recall_score: 0.2505 - f1: 0.2553
Epoch 3/200
 - 0s - loss: 1.0886 - binary_accuracy: 0.7713 - precision_score: 0.3975 - recall_score: 0.2098 - f1: 0.2712
Epoch 4/200
 - 0s - loss: 1.0250 - binary_accuracy: 0.7791 - precision_score: 0.1654 - recall_score: 0.0413 - f1: 0.0661
Epoch 5/200
 - 0s - loss: 1.0474 - binary_accuracy: 0.7791 - precision_score: 0.3385 - recall_score: 0.0840 - f1: 0.1313
Epoch 6/200
 - 0s - loss: 0.9737 - binary_accuracy: 0.7829 - precision_score: 0.3969 - recall_score: 0.0751 - f1: 0.1260
Epoch 7/200
 - 0s - loss: 1.0561 - binary_accuracy: 0.7907 - precision_score: 0.4961 - recall_score: 0.0559 - f1: 0.0992
Epoch 8/200
 - 0s - loss: 0.9731 - binary_accuracy: 0.7791 - precision_score: 0.2894 - recall_score: 0.0370 - f1: 0.0652
Epoch 9/200
 - 0s - loss: 0.9084

 - 0s - loss: 0.7064 - binary_accuracy: 0.9302 - precision_score: 0.9416 - recall_score: 0.6951 - f1: 0.7996
Epoch 69/200
 - 0s - loss: 0.6560 - binary_accuracy: 0.9302 - precision_score: 0.9707 - recall_score: 0.6799 - f1: 0.7910
Epoch 70/200
 - 0s - loss: 0.6733 - binary_accuracy: 0.9264 - precision_score: 0.9096 - recall_score: 0.7169 - f1: 0.8006
Epoch 71/200
 - 0s - loss: 0.6074 - binary_accuracy: 0.9380 - precision_score: 0.9612 - recall_score: 0.7205 - f1: 0.8226
Epoch 72/200
 - 0s - loss: 0.7777 - binary_accuracy: 0.9457 - precision_score: 0.9468 - recall_score: 0.7701 - f1: 0.8487
Epoch 73/200
 - 0s - loss: 0.6284 - binary_accuracy: 0.9380 - precision_score: 0.9516 - recall_score: 0.7540 - f1: 0.8389
Epoch 74/200
 - 0s - loss: 0.6466 - binary_accuracy: 0.9380 - precision_score: 0.9647 - recall_score: 0.7166 - f1: 0.8203
Epoch 75/200
 - 0s - loss: 0.5854 - binary_accuracy: 0.9496 - precision_score: 0.9774 - recall_score: 0.7753 - f1: 0.8644
Epoch 76/200
 - 0s - loss: 0.6424 - b

Epoch 135/200
 - 0s - loss: 0.5916 - binary_accuracy: 0.9612 - precision_score: 0.9922 - recall_score: 0.8020 - f1: 0.8866
Epoch 136/200
 - 0s - loss: 0.6098 - binary_accuracy: 0.9574 - precision_score: 0.9716 - recall_score: 0.8085 - f1: 0.8816
Epoch 137/200
 - 0s - loss: 0.5857 - binary_accuracy: 0.9651 - precision_score: 0.9922 - recall_score: 0.8314 - f1: 0.8998
Epoch 138/200
 - 0s - loss: 0.5932 - binary_accuracy: 0.9612 - precision_score: 0.9816 - recall_score: 0.8246 - f1: 0.8937
Epoch 139/200
 - 0s - loss: 0.5715 - binary_accuracy: 0.9612 - precision_score: 1.0000 - recall_score: 0.8122 - f1: 0.8962
Epoch 140/200
 - 0s - loss: 0.6000 - binary_accuracy: 0.9535 - precision_score: 0.9922 - recall_score: 0.7714 - f1: 0.8680
Epoch 141/200
 - 0s - loss: 0.6036 - binary_accuracy: 0.9535 - precision_score: 0.9751 - recall_score: 0.7886 - f1: 0.8719
Epoch 142/200
 - 0s - loss: 0.5672 - binary_accuracy: 0.9612 - precision_score: 0.9922 - recall_score: 0.8103 - f1: 0.8920
Epoch 143/200
 -

Great! We now have a MLP classifier for collocations. let evaluate it on the val_data_set:

## Evaluating the model

In [6]:
val_dataset = data_set = NpSemanticSegData(val_output_path, train_to_test_ratio=1)

loss, binary_accuracy, precision, recall, f1 = model.eval(val_dataset.train_set)
print('loss = %.1f%%' % (loss))
print('Test binary_accuracy rate = %.1f%%' % (binary_accuracy * 100))
print('Test precision rate = %.1f%%' % (precision * 100))
print('Test recall rate = %.1f%%' % (recall * 100))
print('Test f1 rate = %.1f%%' % (f1 * 100))


loss = 1.3%
Test binary_accuracy rate = 83.3%
Test precision rate = 0.0%
Test recall rate = 0.0%
Test f1 rate = 0.0%
